# Postprocessing

We're going to process some sorting results, and calculate e.g.
- Waveforms
- Templates
- Features, like amplitudes, PCAs etc.

This is good point to take a look at the output, too.

To get started, let's make some simulated data. The following function will make a simulated 60 second recording with 6 channels, and a simulated sorting based on the recording with 10 units and a firing rate of 6 spikes per second.

In [ ]:
import spikeinterface.full as si

recording, sorting = si.generate_ground_truth_recording(
    durations=[
        60.0,
    ],
    sampling_frequency=30_000.0,
    num_channels=6,
    num_units=10,
    generate_sorting_kwargs=dict(firing_rates=6.0, refractory_period_ms=4.0),
    noise_kwargs=dict(noise_levels=5.0, strategy="tile_pregenerated"),
    seed=1205,
)

Now let's combine these into a a single sorting analyzer called `analyzer`. Here, we're saving the sorting analyzer in a folder called "my_sorting_analyzer".

In [ ]:
analyzer = si.create_sorting_analyzer(
    sorting=sorting,
    recording=recording,
    overwrite=True,
    format="binary_folder",
    folder="my_sorting_analyzer"
)


The physical information we're interested in is computed using _extensions_. We can compute the `waveform` extension as follows:

In [ ]:
analyzer.compute("waveforms")

Oh no - an error! The waveforms extensions requres the random_spikes extension. In fact, many extensions depend on other extensions. Here's all the current extension in spike interface and how they depend on one another:

So, we should calculate random_spikes as well as waveforms. In fact we can calculate several extensions in one funciton call. Here we'll calculate random_spikes, waveforms and templates:

In [ ]:
analyzer.compute(["random_spikes","waveforms","templates"])

Since we've saved the sorting analyzer as a folder, the extensions will appear in the folder too. Let's have a look...

...

...

...

Great! We'll now have a look at the waveforms we've calculated. We'll do this using a _widget_. These are used to make graphical, interactive output in Jupyter notebooks. They can be a bit fiddely to set up. For instance, I failed to get them working in VSCode. Others have succeeded. Let's have a go...

In [ ]:
# activate the matplotlib widget
import matplotlib.pyplot
%matplotlib widget

# plot the waveforms
si.plot_unit_waveforms(analyzer, unit_ids=[1,2], figsize=(6,4))

Beautiful! The `plot_unit_waveforms` function plots all the waveforms and the template for that unit. There are _a lot_ of plotting functions: https://spikeinterface.readthedocs.io/en/latest/modules/widgets.html#available-plotting-functions

Let's try another one

In [ ]:
si.plot_unit_summary(analyzer, unit_id=1, figsize=(6,4))

There are many plotting functions which plot things like unit locations and spike amplitudes but we've not calcualted these yet. So let's calculate them! These are also extensions so work in the exact same way as the other extensions we've used.

Since we're going to do lots of calculations, let's set up the calculation using a dictionary. This might suit your style of coding a better. If we give a blank dictionary to the extension name, it will compute the extension using the default parameters

In [ ]:
extensions_to_compute = {
    'principal_components': {
        'n_components': 4
    },
    'spike_amplitudes': {},
    'amplitude_scalings': {},
    'spike_locations': {},
    'template_metrics': {},
    'template_similarity': {},
    'unit_locations': {
        'method': 'monopolar_triangulation'
    },
}

analyzer.compute(extensions_to_compute)

If you ever want to check the kwargs an extension takes, you can use Jupyter's help feature as follows:

In [ ]:
si.compute_spike_amplitudes?

Now that we've calculated some properties, let's take a look at them. You can access the data which each extension hold using the `get_data` method. All of this is funneled through the sorting analyzer object.

In [ ]:
spike_amps = analyzer.get_extension('spike_amplitudes').get_data()
spike_amps

Above are the amplitudes of all the spikes in our recording. Let's try and visualise them ourselves using matplotlib

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
plt.hist(spike_amps, bins=100)
plt.show()

As well as spike information, we also calculated unit features. These are the positions of the units:

In [ ]:
analyzer.get_extension("unit_locations").get_data()

Instead of making our own plotting function, let's use one of the pre-built ones. Again, this uses widgets.

In [ ]:
si.plot_unit_locations(analyzer, figsize=(4,4))

One of these doesn't look very physical: it's located outside of the probe! The `matplotlib` widget is static. Sometimes the dynamic `ipywidgets` widget can be helpful. This is another widget, which works for some plotting functions (again, more details here: https://spikeinterface.readthedocs.io/en/latest/modules/widgets.html#available-plotting-functions)

In [ ]:
si.plot_unit_locations(analyzer, backend="ipywidgets")

So, using this widget we've discovered that unit number 7 looks a bit suspicious. Nice.

# END

That's the end of this notebook. Hopefully you've learned about
- Visualising your sorting results
- Calculating extensions, and their parent/child relationships
- Accessing extension data and visualising them